# Yapay Sinir Ağları

# Model

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [3]:
# Önce veriye dönüştürme işlemi yapmamız gerekiyor

Bir çok katman olduğundan dolayı verideki aykırılıklar ve değişkenlerin varyans ayapılarının birbirinden çok farklı olması elde edilen sonuçların güvenilirliğini düşürmektedir. Bundan dolayı YSA'da özellikle bu işlem yapılmalıdır.

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [6]:
X_train_scaled = scaler.transform(X_train)

In [7]:
X_test_scaled = scaler.transform(X_test)

In [8]:
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.fit_transform(X_test)
# burada fit yapmak ile fit_transform yapmak arasında fark olur muydu, dene bitince

### Şimdi Model Kurulumu zamanı

In [9]:
from sklearn.neural_network import MLPRegressor

In [21]:
mlp_model = MLPRegressor(hidden_layer_sizes = (100,20)).fit(X_train_scaled, y_train)

In [22]:
mlp_model.get_params() # inceleme için

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100, 20),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [23]:
mlp_model.n_layers_ # katman sayısıdır, genel framework (büyük resimi) ifade eder -> (1i, 1o, 1h)

4

In [24]:
mlp_model.hidden_layer_sizes # genel framework'ün gizli katmanlarının içindeki nöron sayısı

(100, 20)

# Tahmin

In [25]:
mlp_model.predict(X_train_scaled)[0:4]

array([ 88.3524026 , 364.90976787, 265.6522415 ,  67.16070214])

In [26]:
y_pred = mlp_model.predict(X_test_scaled)

In [27]:
np.sqrt(mean_squared_error(y_test, y_pred))

455.3815535656345

# Model Tuning

In [28]:
mlp_model.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100, 20),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [29]:
# 'hidden_layer_sizes': (100, 20), kısmı

Bu parametreye girilen sayılar yeni katman eklemesi olarak düşünülür. Girilen sayı da katmanın nöron sayısını ifade eder.

Bu hiperparametre girilen sayıyla beraber 3. katmanı oluşturacağını ve onun kaç nöronlu olacağını anlar.

In [31]:
mlp_params = {"alpha": [0.1, 0.01, 0.02, 0.005], 
              "hidden_layer_sizes": [(20, 20), (100, 50, 150), (200, 300, 150)],
              "activation": ["relu", "logistic"]}

In [33]:
mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv = 10)

In [34]:
mlp_cv_model.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(hidden_layer_sizes=(100, 20)),
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (200, 300, 150)]})

In [35]:
mlp_cv_model.best_params_

{'activation': 'relu', 'alpha': 0.02, 'hidden_layer_sizes': (100, 50, 150)}

In [36]:
# final modeli oluşturma

In [38]:
mlp_tuned = MLPRegressor(alpha = 0.02, activation = "relu", hidden_layer_sizes = (100,50,150))

In [41]:
mlp_tuned.fit(X_train_scaled, y_train) # tune edilmiş modeli oluşturduk optimum hiperparametrelerle

MLPRegressor(alpha=0.02, hidden_layer_sizes=(100, 50, 150))

In [42]:
# son test hatasının oluşturulması ve karşılaştırılması

In [43]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [44]:
np.sqrt(mean_squared_error(y_test, y_pred))

359.1287319734944

### YSA: 359, SVR: 367, KNN: 413, NonlinearSVR : 473
#### Yukarıdaki tune edilmiş RMSE sonuçlarına bakarak en doğru tahminleri yapan modelimizin YSA olduğunu söyleyebiliriz